In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## 1) Loading Googles API key

In [8]:
google_api_key = os.getenv("GEMINI_API_KEY")#using thise api key u can access any gen ai model of google,,,untilll free tier is completed OR amount is finished from account

In [9]:
if google_api_key == "" :
    print('api key Not-found')
else:
    print('api key found')

api key found


## 2) Importing Required Libraries

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini 
from IPython.display import Markdown, display 
from llama_index.core import ServiceContext 
from llama_index.core import StorageContext, load_index_from_storage 
import google.generativeai as genai 
from llama_index.embeddings.gemini import GeminiEmbedding


d:\GENERATIVE-AI\007_Projects\002_QA_System_Using_Llama_Index_And_Gemini\qa_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Llama index provides the key tools to augment your llm app with data
  1) Data Ingestion: connect your existing data sources and data formats (3rd party API , PDF's , Documents , SQL ,NoSQL..etc) to use with a LLM model application
  2) Data Indexing: store and index your data for different use cases.Integrate with downstream vector store and DB providers
  3) Query Interface: provides query interface that accepts any input prompt over your data and returns a knowledge-augmented responses

- Main purpose of Llama index is,**storing and retrieving vectors** 
- LangChain has many more functionality compare to this.ie; **chains and memory**.And used for developing end-to-end genai application.
- So,u can Use  both to develop your application stronlgy...
- but no need to separetly import llama index and langchain........bcz llama index has integrated with langchain......so,,just import llama index,,from this you can access langchain functionality wherever you want...........
- llama index has support of langchain,ie;inside llama index there is a langchain functionality also,which we are going to access through llama index

## 3) connect with Google gen ai models services

In [10]:
genai.configure(api_key = google_api_key )
#This genai alias for python SDK for accessing google's gemini models....and we setted up the api_key internally,,,
#thus while accessing google gemini model , embedding model,,no need to write this_api key,,,,internally inferred

In [11]:
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
         print(model.name)
#These are Generation based models

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash


## 4) Create Data Folder and add files on which you want to perform QA

- I have added ML_Wikipedia.txt,in which 2329 words(12208 character are there) are there according to openai tokenizer..approx 4 character means 1 word
- if you give this file to google tokenization then it will give you tokens inside this file,,,,,more or less token counts are nearer
- Add any no and any kind of file you want,,,

## 5) Load text present in all the files ,which are present in Data folder

In [4]:
documents = SimpleDirectoryReader("../Data")
#i am inside notebook folder,and Data folder is present outside of this folder and inside of 002_QA.......
#so from current folder (ie;when we are executing experiments.ipynb then we are in 'notebook' directory) come outside,,and select Data folder from it

In [8]:
doc = documents.load_data()
doc

[Document(id_='458acf2b-1243-4798-b6bd-8ef62822e80f', embedding=None, metadata={'file_path': 'd:\\GENERATIVE-AI\\007_Projects\\002_QA_System_Using_Llama_Index_And_Gemini\\notebook\\..\\Data\\ML_Wikipedia.txt', 'file_name': 'ML_Wikipedia.txt', 'file_type': 'text/plain', 'file_size': 12283, 'creation_date': '2024-07-31', 'last_modified_date': '2024-07-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\r\n

In [9]:
doc[0]

Document(id_='458acf2b-1243-4798-b6bd-8ef62822e80f', embedding=None, metadata={'file_path': 'd:\\GENERATIVE-AI\\007_Projects\\002_QA_System_Using_Llama_Index_And_Gemini\\notebook\\..\\Data\\ML_Wikipedia.txt', 'file_name': 'ML_Wikipedia.txt', 'file_type': 'text/plain', 'file_size': 12283, 'creation_date': '2024-07-31', 'last_modified_date': '2024-07-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\r\n\

In [10]:
print(doc[0].text)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

## 6) partition Above Data into chunks and perform embeddings on each chunk and store it in local vectorstore

- models/text-embedding-004 
- models/embedding-001
- these 2 embedding models we have...
- use any one,,,these are free of cost,i will use latest one......text-emnedding-004

In [11]:
gemini_embed_model = GeminiEmbedding(model_name='models/text-embedding-004')#Internally access the API (google_api_key)

In [12]:
gemini_embed_model

GeminiEmbedding(model_name='models/text-embedding-004', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000192CD4F0050>, num_workers=None, title=None, task_type='retrieval_document', api_key=None)

In [13]:
# import google.generativeai as genai -> accessing gemini models through python SDK...python library
# from llama_index.llms.gemini import Gemini -> accessing gemini model through llama index frame work,which internally has this python SDK implementation.
#Both are same thing,,ie;import google.generativeai as genai this is python SDk.through which u can access gemini
#from llama_index.llms.gemini import Gemini , this  llama index also has this python SDk,,which internally uses this SDK to call gemini
#You can use any one of them,both are same only,,,,,,,,,,,

In [14]:
gemini_pro_model = Gemini(model = 'models/gemini-pro',api_key = google_api_key )

### Service Context

- The service context container is a utility container for LlamaIndex index and query classes. The container contains the following objects that are commonly used for configuring every index and query, such as the LLM, the PromptHelper (for configuring input size/chunk size), the BaseEmbedding (for configuring the embedding model), and more.
- Service Context Classes
    1) Embeddings
    2) OpenAIEmbedding
    3) HuggingFaceEmbedding
    4) OptimumEmbedding
    5) InstructorEmbedding
    6) LangchainEmbedding
    7) GoogleUnivSentEncoderEmbedding
    8) Node Parser
    9) PromptHelper
    10) 
- **conclusion** -> for storing and indexing the vectors/embeddings......and to perform Similarity search opration on embedding w.r.t query to get most similar vector,,,,And passing it to specified LLM model to get fine tuned answer

In [15]:
# Configure Service Context object
service_context = ServiceContext.from_defaults(llm=gemini_pro_model , embed_model=gemini_embed_model , chunk_size=800 , chunk_overlap=20)

C:\Users\shree\AppData\Local\Temp\ipykernel_20840\1479180934.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=gemini_pro_model , embed_model=gemini_embed_model , chunk_size=800 , chunk_overlap=20)


In [16]:
index = VectorStoreIndex.from_documents(doc , service_context=service_context)#deviding our above data in to chunk,,then each chunk is vectorized,,and stored in index..

- first chunk is accessed -> pass it to embedding model(google_embedding_model)ie;models/text-embedding-004 -> it will return single vector/embedding with 768 dimensions -> and finally this vector/embedding for this chunk and this actual chunk(in text format) as meta data is going to store in index object,,,which is VectorStore DB,,,,,,,
- same for all remaining chunks
- Why we are storing chunks(text) along with there vectors/embedding ?
- bcz,,when you query something,,,then ,,that query (obviously it is string/text) will first converted to vector/embedding -> passed to this vectordb (index) for similarity match -> and return most similar vector present in index(vectorstore db) with query vector -> It will not return that vector/embedding,,instead return respective chunk to that vector.............
- bcz ,,we have aggregated the words of chunks to get single vector for chunk,,,,,,,,,,,this process is irreversible,,,,,from this vector your cant get actual chunk,,,,,(only,,word to vector and vector to word is possible),,,,,,,vector of the chuk cant be converted back to actual chunk..bcz after aggregating all word's repective vector to get single vector representing chunk,we lose info,,that what are the words,,so

- After returning this chunk,,,(as u see in servicecontext,, chunk contains approx 800 words/tokens in it after splitting/partition),,,we want specific info from this w.r.t question,,so to refine the response according to query,,we pass this chunk ,query to specified LLM,,and we get refined or specif answer/response

- **Conclusion**-> Vector/embeddings are just for representing each chunk and perform similaity search and other vector operation on it,,to get desired vector,,,,,,,,,,,but we cant understand this vector,,,so we store ,,chun(text) along with its vector form..........so while retrieving,,we getjust chunk,,,,and performing similarity search we need just vector.

In [26]:
index

In [17]:
#To persist this embeddings/vector of each chunk
index.storage_context.persist()
#as u see,,storage is created,,which is local vetor DB,,bcz we have not mention any remote db api_key..........if u  mention pinecone or any other cloud based vector db api_key,,then these vectors are transfeered to that DB....
#inside this storage folder......you see lot of meta data..............but you cant see our embeddings related to ML_Wikipedia
#bcz it is stored in SQLLite,,,,,,in binary obj,so you just access/retrieve chun(text) w.r.t vector,,,u can't see these vectros/embeddings for each chunk,,,as u see in pinecone Db

In [28]:
#see this storage folder,,and try to inerpret it.

## 7) Query on Vector store ie; Questioning some data from ML_Wikipedia file to vectorstoe(index),,QUERY ON DOCUMENT Given document

- We have given just ML_Wikipedia.txt file,,,,see in Data folder,,,we have just this file,,,and thus just this file is partioned to chunk -> converted to embedding/vector for each chunk and stored in index..
- you can add more files,,related to your use case,,,
- **So you can query/question related to this ML_Wikpedia file only,,,,,,,,,,,,,,,,,you cant query other things which are not present in this file**

#### Q1)Querying based on stored document (ML_Wikipedia)

In [18]:
query_engine = index.as_query_engine()

In [19]:
response = query_engine.query("Which Machine Learning Model we Use in Data Compression?")

In [20]:
response 

Response(response='K-means clustering', source_nodes=[NodeWithScore(node=TextNode(id_='c7ac529a-8575-4fd2-be85-1ef03c6ebeb5', embedding=None, metadata={'file_path': 'd:\\GENERATIVE-AI\\007_Projects\\002_QA_System_Using_Llama_Index_And_Gemini\\notebook\\..\\Data\\ML_Wikipedia.txt', 'file_name': 'ML_Wikipedia.txt', 'file_type': 'text/plain', 'file_size': 12283, 'creation_date': '2024-07-31', 'last_modified_date': '2024-07-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='458acf2b-1243-4798-b6bd-8ef62822e80f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\GENERATIVE-AI\\007_Projects\\002_QA_System_Using_Llama_Index_And_Gemini\\notebook\\..\\Data\\ML_Wikipedia.txt', 'file_name': 'ML_Wiki

In [21]:
response.response #print(response)
#Read,,,,,,ML_Wikipedia file,,,in data compression part,,,you see,,correct answer is KNN.............
#This is power Of RAG

'K-means clustering'

- See text field from response,,this is resposne we got from vectorDb (index),,based on similarity score b/n query vector and store vector.......as we discussed,we got complete chunk w.r.t to matched vector
- this chunk/text contains approx 800 words,,,and this chunk contains our answer,,but we have to do manuall work or read this text to get specific answer w.r.t our question,,,,,,which is difficult task when we deal with large chunk_size's,,,,,,,,
- **Recoomened Way**-> Pass this retrieved text/chunk to any LLM(not passing vector,,see diagram of RAG in book) along with query,,and then llm will extract particulr data from this text w.r.t query/question and thus return specific answer.....**This Process is Called RAG**


#### Q2) Query which is not based on stored document (ML_Wikipedia)...ie;which is irrelevant to passed/stored/used document (ML_wikipedia)

In [41]:
response2 = query_engine.query("Who is Mahendr singh dhoni?")

In [43]:
response2

Response(response='The provided context does not mention Mahendr Singh Dhoni, so I cannot answer this question from the provided context.', source_nodes=[NodeWithScore(node=TextNode(id_='f70488d8-3e2c-442c-8c2a-85f02a5c59c1', embedding=None, metadata={'file_path': 'd:\\GENERATIVE-AI\\007_Projects\\002_QA_System_Using_Llama_Index_And_Gemini\\notebook\\..\\Data\\ML_Wikipedia.txt', 'file_name': 'ML_Wikipedia.txt', 'file_type': 'text/plain', 'file_size': 12283, 'creation_date': '2024-07-31', 'last_modified_date': '2024-07-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='19e331ad-4f73-475f-8ec3-a039c9cbe94d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\GENERATIVE-AI\\007_Projects\\002_

In [42]:
response2.response #print(response2)

'The provided context does not mention Mahendr Singh Dhoni, so I cannot answer this question from the provided context.'

- We are not getting answer..................bcz,,we have passed and stored ML_Wikipedia.txt file in our application....and we stored the vectors or embedding for them in index(vectro store db)
- An when you read this file carefully,,then you observe that this file not contain "mahendr singh dhoni" words or data related to msd,,,,,,,,
- thus,vectorstore(index),,,doesnt have similar vectors w.r.t query vector,,,,,,,,,,,,thus similarity score of each strored vector with query vector is in negative....and return the the chunk whose equivavlent vector has more similarity score compare to other (by the way it is in negative,,tecnically,,no similar,,,,but returning most similar vector),,,and when ,,,query_engine pass this text/chunk and query to specified LLM(in our case it is gemini-pro),,,then that LLM will see,,this text and come to know that,,this chunk doesnt have answer to given query,,,,,,so it will return 'The provided context does not .......'............
- **Note:** Here or in RAG system ,,role of LLM is just to refine/get specific info from passed chunk w.r.t query,,,,,,,,,,it will not provide answer by its own when it doesnt find any answer in passed chunk/text,,as it does while normal chating (like,,chatgpt,gemini application)
- **Conclusion:** When query_engine will pass text and query to LLM,,,,LLM will analyze the text w.r.t query and return specific answer which is refined answer,,,,,,,,,,,,,,,,,,,in the case of failure in finding answer to query in given text,,it will simply return 'This provided....message',it will not give answer by itself...........................which is good thing.................we are getting only relevant info from our data,,,not from external sources..................
- but if u want answer on failure case also,,then mention to LLM that,,if not get the answer then give answer according to your knowledge

#### For Any question/query and text/chunk passed by query_engine to LLM,,LLM will see the passed chunk/text only to extract answers,,,if it finds then return it, else return 'not find in passed document'

- This is our required experimentation,,,now we designed a rough software,,which align with our requiremnt functinality,,,,,,,,,now do modular coding and use OOP's to segrate the code according to there functionality, thus to increase readability,code managability, updation ..etc
- then,, integrate these thing in our end-to-end application pipeline,,to get industry ready/deployment ready application

- **bcz this vectorestore stores the data,,localy,so while deploying this app on web,,,we use ec2 instance and in its storage only,,we store the embeddings**

In [ ]:
### ThankYou